In [10]:
# 📂 SNAPSHOT IMPORT - Campaign Statistics & Stock Data
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# 📁 Snapshot Directory
snap_dir = Path(r"C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\snapshots")
print(f"📁 SNAP dir: {snap_dir}")

# 🧊 Import Campaign Statistics Snapshot
campaign_stats_path = snap_dir / "campaign_statistics.pkl"
campaign_stats = pd.read_pickle(campaign_stats_path)
print(f"🧊 Snapshot rebuilt: campaign_statistics.pkl  ({campaign_stats.shape[0]}×{campaign_stats.shape[1]})")

# 🧊 Import Detailed Stock List Snapshot  
stock_path = snap_dir / "detailed_stock_list.pkl"
stock_data = pd.read_pickle(stock_path)
print(f"🧊 Snapshot rebuilt: detailed_stock_list.pkl  ({stock_data.shape[0]}×{stock_data.shape[1]})")

print(f"\n✅ Snapshots loaded successfully")
print(f"📊 Analysis ready: {datetime.now().strftime('%B %d, %Y at %H:%M:%S')}")

📁 SNAP dir: C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\snapshots
🧊 Snapshot rebuilt: campaign_statistics.pkl  (28×23)
🧊 Snapshot rebuilt: detailed_stock_list.pkl  (4714×43)

✅ Snapshots loaded successfully
📊 Analysis ready: October 08, 2025 at 11:34:51


In [14]:
# 🎨 COLOR-CODED TOP 25 WINNERS BY WINE PRICE
from IPython.display import display

def format_swiss_number(number):
    """Format numbers in Swiss style with apostrophes (82'723.98)"""
    if pd.isna(number) or number == 0:
        return "0.00"
    
    # Round to 2 decimal places
    number = round(float(number), 2)
    
    # Split into integer and decimal parts
    integer_part = int(abs(number))
    decimal_part = f"{abs(number) - integer_part:.2f}"[2:]  # Get decimal part (2 digits)
    
    # Format integer part with apostrophes
    formatted_int = f"{integer_part:,}".replace(",", "'")
    result = f"{formatted_int}.{decimal_part}"
    
    return f"-{result}" if number < 0 else result

print("🎨 COLOR-CODED TOP 25 WINNERS BY WINE PRICE")
print("="*55)
print(f"📊 Analysis Date: {datetime.now().strftime('%B %d, %Y at %H:%M:%S')}")
print("🏆 Winner Calculation: 60% Conversion Rate + 40% Total Sales Amount")
print("❌ Excluded: Type=Horeca/Trade, Sub-Type=Lead")
print("📊 Normalization: Norm_Conversion = conversion_rate/max_conversion | Norm_Sales = total_sales/max_sales")
print("💰 Swiss Formatting: Total Sales with apostrophes (82'723.98)\n")

# ---- Filter Campaign Statistics ----
campaigns = campaign_stats.copy()
print(f"📊 Total campaigns loaded: {len(campaigns)}")

# Apply filters using correct column names
campaigns_filtered = campaigns[
    (~campaigns['type'].isin(['Horeca', 'Trade'])) &
    (campaigns['sub-type'] != 'Lead')
].copy()

print(f"📊 After filtering (excluding Horeca/Trade & Lead): {len(campaigns_filtered)}")

# ---- Extract key columns for winner calculation ----
# Column references based on the actual data structure
conversion_rate = pd.to_numeric(campaigns_filtered['conversion rate %'], errors='coerce').fillna(0)
total_sales = pd.to_numeric(campaigns_filtered['total sales amount (lcy)'], errors='coerce').fillna(0)

# Other display columns
wine_name = campaigns_filtered['main wine name'].fillna('Unknown')
campaign_no = campaigns_filtered['campaign no.'].fillna('')
vintage_code = campaigns_filtered['vintage code'].fillna('')
scheduled_datetime = campaigns_filtered['scheduled datetime1'].fillna('')  # Using scheduled datetime1
multiple_wines = campaigns_filtered['multiple wines'].fillna('')
email_sent = pd.to_numeric(campaigns_filtered['email sent'], errors='coerce').fillna(0)
main_bottle_price = pd.to_numeric(campaigns_filtered['main bottle price (lcy)'], errors='coerce').fillna(0)
delayed_sending = campaigns_filtered['delayed sending'].fillna(False)

# ---- Calculate Winner Score (60% conversion + 40% sales) ----
# Normalize conversion rate and sales for scoring
max_conversion = max(conversion_rate.max(), 1e-12)  # Avoid division by zero
max_sales = max(total_sales.max(), 1e-12)

print(f"📊 Normalization factors: Max Conversion = {max_conversion:.2f}% | Max Sales = CHF {format_swiss_number(max_sales)}")

# Calculate normalized scores
norm_conversion = conversion_rate / max_conversion  # Each campaign's conversion / best conversion
norm_sales = total_sales / max_sales  # Each campaign's sales / best sales

# Final weighted score: 60% conversion + 40% sales
weighted_score = 0.6 * norm_conversion + 0.4 * norm_sales

# ---- Build Winners DataFrame ----
winners_df = pd.DataFrame({
    'Campaign_No': campaign_no,
    'Wine': wine_name,  # Renamed from Main_Wine_Name to Wine
    'Vintage': vintage_code,  # Renamed from Vintage_Code to Vintage
    'Starting_Date': scheduled_datetime,
    'Multiple': multiple_wines,
    'Email_Sent': email_sent,
    'Conversion_Rate_%': conversion_rate,
    'Total_Sales_Amount_LCY': total_sales,
    'Norm_Conversion': norm_conversion,
    'Norm_Sales': norm_sales,
    'Weighted_Score': weighted_score,
    'Main_Bottle_Price_LCY': main_bottle_price,
    'Delayed_Sending': delayed_sending
})

# Sort by weighted score (descending) and get top 25
winners_df = winners_df.sort_values('Weighted_Score', ascending=False).reset_index(drop=True)
top_25_winners = winners_df.head(25).copy()

# ---- Price Category Color Coding ----
def get_price_emoji(price):
    """Return emoji based on price tier"""
    if pd.isna(price) or price <= 0: return "⚪"
    if price >= 1000: return "🟣"  # Extra Luxury
    if price >= 500:  return "🟨"  # Luxury
    if price >= 150:  return "🟦"  # Premium
    if price >= 80:   return "🩷"  # Mid-Range
    return "🟢"  # Budget

# Add color coding and rank (on the right)
top_25_winners['🎨'] = top_25_winners['Main_Bottle_Price_LCY'].apply(get_price_emoji)
top_25_winners['Rank'] = range(1, len(top_25_winners) + 1)

# ---- Display Results ----
print("🎨 COLOR LEGEND:")
print("🟣 Purple: Extra Luxury (1000+ CHF)")
print("🟨 Gold:   Luxury (500+ CHF)")
print("🟦 Blue:   Premium (150+ CHF)")
print("🩷 Pink:   Mid-Range (80+ CHF)")
print("🟢 Green:  Budget (<80 CHF)")
print("⚪ White:  Unknown/No price\n")

# Prepare display table with requested columns order
display_table = top_25_winners[['Campaign_No', '🎨', 'Wine', 'Vintage', 'Starting_Date',
                               'Multiple', 'Email_Sent', 'Conversion_Rate_%', 'Total_Sales_Amount_LCY',
                               'Norm_Conversion', 'Norm_Sales', 'Weighted_Score', 'Delayed_Sending', 'Rank']].copy()

# Format wine names for better display
display_table['Wine'] = display_table['Wine'].astype(str).apply(
    lambda x: x if len(x) <= 25 else x[:25] + "…"
)

# Format vintage to show just the year number (remove .0)
display_table['Vintage'] = pd.to_numeric(display_table['Vintage'], errors='coerce').fillna(0).astype(int).astype(str).replace('0', '')

# Format starting date for better display (remove time, show only date)
display_table['Starting_Date'] = pd.to_datetime(display_table['Starting_Date'], errors='coerce').dt.strftime('%Y-%m-%d').fillna('')

# Add "-D" suffix to Campaign_No if Delayed_Sending is True
display_table['Campaign_No'] = display_table.apply(
    lambda row: f"{row['Campaign_No']}-D" if row['Delayed_Sending'] == True else str(row['Campaign_No']), 
    axis=1
)

# Format Total_Sales_Amount_LCY with Swiss formatting (2 decimal places)
display_table['Total_Sales_Amount_LCY'] = display_table['Total_Sales_Amount_LCY'].apply(format_swiss_number)

# Round numeric columns for better display
display_table['Norm_Conversion'] = display_table['Norm_Conversion'].round(4)
display_table['Norm_Sales'] = display_table['Norm_Sales'].round(4)
display_table['Weighted_Score'] = display_table['Weighted_Score'].round(4)

# Remove Delayed_Sending column from display (used only for formatting)
display_table = display_table.drop('Delayed_Sending', axis=1)

print("🏆 TOP 25 WINE CAMPAIGN WINNERS:")
display(display_table)

# ---- Summary Statistics ----
print("\n📊 WINNER ANALYSIS SUMMARY:")
print(f"• #1 Winner: {top_25_winners.iloc[0]['Wine']} | Weighted Score: {top_25_winners.iloc[0]['Weighted_Score']:.4f}")
print(f"• Price Range: CHF {top_25_winners['Main_Bottle_Price_LCY'].min():.0f} - CHF {top_25_winners['Main_Bottle_Price_LCY'].max():.0f}")
print(f"• Avg Conversion Rate: {top_25_winners['Conversion_Rate_%'].mean():.2f}%")
print(f"• Total Sales (Top 25): CHF {format_swiss_number(top_25_winners['Total_Sales_Amount_LCY'].sum())}")
print(f"• Total Emails Sent: {top_25_winners['Email_Sent'].sum():,.0f}")
print(f"• Avg Normalized Conversion: {top_25_winners['Norm_Conversion'].mean():.4f}")
print(f"• Avg Normalized Sales: {top_25_winners['Norm_Sales'].mean():.4f}")
print(f"• Delayed Campaigns: {(top_25_winners['Delayed_Sending'] == True).sum()}/{len(top_25_winners)}")

# Price category distribution
print(f"\n🎨 PRICE CATEGORY DISTRIBUTION:")
price_categories = top_25_winners['🎨'].value_counts()
price_meanings = {
    "🟣": "Extra Luxury (1000+ CHF)",
    "🟨": "Luxury (500+ CHF)", 
    "🟦": "Premium (150+ CHF)",
    "🩷": "Mid-Range (80+ CHF)",
    "🟢": "Budget (<80 CHF)",
    "⚪": "Unknown/No price"
}

for emoji, count in price_categories.items():
    meaning = price_meanings.get(emoji, "Unknown")
    pct = 100 * count / len(top_25_winners)
    print(f"   {emoji} {meaning}: {count} campaigns ({pct:.1f}%)")

🎨 COLOR-CODED TOP 25 WINNERS BY WINE PRICE
📊 Analysis Date: October 08, 2025 at 12:12:24
🏆 Winner Calculation: 60% Conversion Rate + 40% Total Sales Amount
❌ Excluded: Type=Horeca/Trade, Sub-Type=Lead
📊 Normalization: Norm_Conversion = conversion_rate/max_conversion | Norm_Sales = total_sales/max_sales
💰 Swiss Formatting: Total Sales with apostrophes (82'723.98)

📊 Total campaigns loaded: 28
📊 After filtering (excluding Horeca/Trade & Lead): 26
📊 Normalization factors: Max Conversion = 4.94% | Max Sales = CHF 82'723.98
🎨 COLOR LEGEND:
🟣 Purple: Extra Luxury (1000+ CHF)
🟨 Gold:   Luxury (500+ CHF)
🟦 Blue:   Premium (150+ CHF)
🩷 Pink:   Mid-Range (80+ CHF)
🟢 Green:  Budget (<80 CHF)
⚪ White:  Unknown/No price

🏆 TOP 25 WINE CAMPAIGN WINNERS:


,Campaign_No,🎨,Wine,Vintage,Starting_Date,Multiple,Email_Sent,Conversion_Rate_%,Total_Sales_Amount_LCY,Norm_Conversion,Norm_Sales,Weighted_Score,Rank
0,CM-25-02630,🟨,IX Proprietary Red Estate,2022,2025-10-02,1,405,4.94,82'723.98,1.0000,1.0000,1.0000,1
1,CM-25-02637-D,🟨,Hommage à Jacques Perrin,2013,2025-10-03,5,28,3.57,1'587.81,0.7227,0.0192,0.4413,2
2,CM-25-02629,🟦,Lynch Bages,2009,2025-10-01,1,2311,0.87,48'449.89,0.1761,0.5857,0.3399,3
3,CM-25-02628-D,🟦,Grillet Blanc,2019,2025-10-02,4,197,2.03,6'674.44,0.4109,0.0807,0.2788,4
4,CM-25-02625-D,🟦,El Tamboril Blanco,2016,2025-10-01,12,170,1.76,9'253.45,0.3563,0.1119,0.2585,5
5,CM-25-02687,🟢,Rieussec La Metamorphose,2019,2025-10-07,1,2107,1.52,11'431.52,0.3077,0.1382,0.2399,6
6,CM-25-02624,🩷,Tignanello,2022,2025-10-01,1,1907,1.00,23'211.00,0.2024,0.2806,0.2337,7
7,CM-25-02645,🟢,Clos Haut Peyraguey,2017,2025-10-03,1,218,1.83,1'398.32,0.3704,0.0169,0.2290,8
8,CM-25-02644,🟦,Yquem,2015,2025-10-03,1,1249,0.56,18'809.76,0.1134,0.2274,0.1590,9
9,CM-25-02582,🟨,Champagne Brut Dom Pérign…,2013,2025-10-01,1,2102,0.48,17'613.31,0.0972,0.2129,0.1435,10



📊 WINNER ANALYSIS SUMMARY:
• #1 Winner: IX Proprietary Red Estate | Weighted Score: 1.0000
• Price Range: CHF 0 - CHF 60000
• Avg Conversion Rate: 0.82%
• Total Sales (Top 25): CHF 254'300.21
• Total Emails Sent: 27,488
• Avg Normalized Conversion: 0.1656
• Avg Normalized Sales: 0.1230
• Delayed Campaigns: 3/25

🎨 PRICE CATEGORY DISTRIBUTION:
   🟦 Premium (150+ CHF): 10 campaigns (40.0%)
   🟨 Luxury (500+ CHF): 6 campaigns (24.0%)
   🟢 Budget (<80 CHF): 5 campaigns (20.0%)
   🩷 Mid-Range (80+ CHF): 2 campaigns (8.0%)
   🟣 Extra Luxury (1000+ CHF): 1 campaigns (4.0%)
   ⚪ Unknown/No price: 1 campaigns (4.0%)


In [13]:
# 📅 MULTI-PERIOD WINNERS ANALYSIS WITH STOCK AVAILABILITY
from IPython.display import display
from datetime import datetime, timedelta

print("📅 MULTI-PERIOD WINNERS ANALYSIS WITH STOCK AVAILABILITY")
print("="*65)
print(f"📊 Analysis Date: {datetime.now().strftime('%B %d, %Y at %H:%M:%S')}")
print("🎯 Period Analysis: Last 7, 14, 21, and 30 days")
print("📦 Stock Status: Based on Detailed Stock List (Column B)")
print("🏆 Winner Logic: 60% Conversion + 40% Sales (filtered by period)\n")

# ---- Stock Processing ----
print("📦 Processing Stock Data...")
stock_clean = stock_data.copy()

# Extract item ID from 'id' column and stock quantity from 'stock' column
stock_mapping = pd.DataFrame({
    'item_id': pd.to_numeric(stock_clean['id'], errors='coerce').fillna(0).astype(int),  # 'id' column as integer
    'stock_quantity': pd.to_numeric(stock_clean['stock'], errors='coerce').fillna(0)  # 'stock' column (quantity)
}).drop_duplicates(subset=['item_id'])

print(f"✅ Stock data processed: {len(stock_mapping)} unique items")
print(f"📊 Stock range: {stock_mapping['stock_quantity'].min():.0f} - {stock_mapping['stock_quantity'].max():.0f} bottles")

# ---- Stock Status Functions ----
def get_stock_emoji(quantity):
    """Return emoji based on stock quantity"""
    if pd.isna(quantity):
        return "⚪"  # Unknown
    elif quantity >= 11:
        return "🟢"  # Good for resending
    elif quantity >= 1:
        return "🟡"  # Proceed with caution
    else:
        return "🔴"  # Avoid resending

def get_stock_status(quantity):
    """Return stock status description"""
    if pd.isna(quantity):
        return "Unknown status"
    elif quantity >= 11:
        return f"Good ({int(quantity)} bottles)"
    elif quantity >= 1:
        return f"Caution ({int(quantity)} bottles)"
    else:
        return "Avoid (0 bottles)"

# ---- Price Category Functions (reuse from cell 2) ----
def get_price_emoji(price):
    """Return emoji based on price tier"""
    if pd.isna(price) or price <= 0: return "⚪"
    if price >= 1000: return "🟣"  # Extra Luxury
    if price >= 500:  return "🟨"  # Luxury
    if price >= 150:  return "🟦"  # Premium
    if price >= 80:   return "🩷"  # Mid-Range
    return "🟢"  # Budget

# ---- Period Analysis ----
current_date = datetime.now()
periods = [
    (7, "Last 7 Days", "🗓️"),
    (14, "Last 14 Days", "📅"), 
    (21, "Last 21 Days", "📆"),
    (30, "Last 30 Days", "🗓️")
]

print("📦 STOCK STATUS LEGEND:")
print("🟢 Green: 11+ bottles (good for resending campaigns)")
print("🟡 Yellow: 1-10 bottles (proceed with caution)")
print("🔴 Red: 0 bottles (avoid resending)")
print("⚪ White: Unknown status (item not found in stock list)\n")

print("🎨 PRICE TIER LEGEND:")
print("🟣 Purple: Extra Luxury (1000+ CHF) | 🟨 Gold: Luxury (500+ CHF)")
print("🟦 Blue: Premium (150+ CHF) | 🩷 Pink: Mid-Range (80+ CHF) | 🟢 Green: Budget (<80 CHF)\n")

# Use the winners data from cell 2 
winners_with_dates = top_25_winners.copy()

# Convert Starting_Date to datetime for filtering
winners_with_dates['Starting_Date_dt'] = pd.to_datetime(winners_with_dates['Starting_Date'], errors='coerce')

# Add stock information by matching main item numbers
# First, we need to get main item numbers from the original data
campaigns_with_items = campaigns_filtered.copy()
item_mapping = pd.DataFrame({
    'Campaign_No': campaigns_with_items['campaign no.'],
    'Main_Item_No': pd.to_numeric(campaigns_with_items['main item no.'], errors='coerce').fillna(0).astype(int)
})

# Merge with winners to get item numbers
winners_with_items = winners_with_dates.merge(
    item_mapping, 
    left_on='Campaign_No', 
    right_on='Campaign_No', 
    how='left'
)

# Match with stock data
winners_with_stock = winners_with_items.merge(
    stock_mapping,
    left_on='Main_Item_No',
    right_on='item_id',
    how='left'
)

# For each period, create analysis
for days, period_name, emoji in periods:
    print(f"\n{emoji} {period_name.upper()} ANALYSIS")
    print("-" * 50)
    
    # Filter by date range
    cutoff_date = current_date - timedelta(days=days)
    period_data = winners_with_stock[
        winners_with_stock['Starting_Date_dt'] >= cutoff_date
    ].copy()
    
    if period_data.empty:
        print(f"⚠️ No campaigns found in {period_name.lower()}")
        continue
    
    # Sort by weighted score and take top 10 for each period
    period_top = period_data.sort_values('Weighted_Score', ascending=False).head(10).copy()
    period_top['Period_Rank'] = range(1, len(period_top) + 1)
    
    # Add stock status
    period_top['📦'] = period_top['stock_quantity'].apply(get_stock_emoji)
    period_top['Stock_Status'] = period_top['stock_quantity'].apply(get_stock_status)
    
    # Prepare display table
    display_cols = [
        'Period_Rank', '🎨', '📦', 'Campaign_No', 'Wine', 'Vintage', 'Starting_Date',
        'Total_Sales_Amount_LCY', 'Conversion_Rate_%', 'Weighted_Score', 'Stock_Status', 'Main_Item_No'
    ]
    
    period_display = period_top[[col for col in display_cols if col in period_top.columns]].copy()
    
    # Format columns
    if 'Total_Sales_Amount_LCY' in period_display.columns:
        period_display['Total_Sales_Amount_LCY'] = period_display['Total_Sales_Amount_LCY'].apply(format_swiss_number)
    
    if 'Weighted_Score' in period_display.columns:
        period_display['Weighted_Score'] = period_display['Weighted_Score'].round(4)
    
    if 'Conversion_Rate_%' in period_display.columns:
        period_display['Conversion_Rate_%'] = period_display['Conversion_Rate_%'].round(2)
    
    # Format Starting_Date to show only date (no time)
    if 'Starting_Date' in period_display.columns:
        period_display['Starting_Date'] = pd.to_datetime(period_display['Starting_Date'], errors='coerce').dt.strftime('%Y-%m-%d').fillna('')
    
    # Format Vintage to show only year (remove .0)
    if 'Vintage' in period_display.columns:
        period_display['Vintage'] = pd.to_numeric(period_display['Vintage'], errors='coerce').fillna(0).astype(int).astype(str).replace('0', '')
    
    # Add -D suffix to Campaign_No if Delayed_Sending is True
    if 'Campaign_No' in period_display.columns and 'Delayed_Sending' in period_top.columns:
        period_display['Campaign_No'] = period_top.apply(
            lambda row: f"{row['Campaign_No']}-D" if row['Delayed_Sending'] == True else str(row['Campaign_No']), 
            axis=1
        )
    
    print(f"🏆 TOP 10 WINNERS - {period_name.upper()}:")
    display(period_display)
    
    # Period summary
    total_campaigns = len(period_data)
    good_stock = (period_top['📦'] == '🟢').sum()
    caution_stock = (period_top['📦'] == '🟡').sum()
    avoid_stock = (period_top['📦'] == '🔴').sum()
    unknown_stock = (period_top['📦'] == '⚪').sum()
    
    print(f"\n📊 {period_name.upper()} SUMMARY:")
    print(f"• Total campaigns in period: {total_campaigns}")
    print(f"• Top 10 campaigns analyzed")
    print(f"• Average weighted score: {period_top['Weighted_Score'].mean():.4f}")
    print(f"• Total sales (Top 10): CHF {format_swiss_number(period_top['Total_Sales_Amount_LCY'].sum())}")
    
    print(f"\n📦 STOCK DISTRIBUTION (Top 10):")
    if good_stock > 0:
        print(f"   🟢 Good for resending: {good_stock} campaigns ({100*good_stock/len(period_top):.1f}%)")
    if caution_stock > 0:
        print(f"   🟡 Proceed with caution: {caution_stock} campaigns ({100*caution_stock/len(period_top):.1f}%)")
    if avoid_stock > 0:
        print(f"   🔴 Avoid resending: {avoid_stock} campaigns ({100*avoid_stock/len(period_top):.1f}%)")
    if unknown_stock > 0:
        print(f"   ⚪ Unknown status: {unknown_stock} campaigns ({100*unknown_stock/len(period_top):.1f}%)")

print("\n🎯 MULTI-PERIOD ANALYSIS COMPLETE")
print("="*40)
print("📊 Use the stock status indicators to make resending decisions")
print("🟢 Green campaigns are ideal for immediate resending")
print("🟡 Yellow campaigns should be monitored closely") 
print("🔴 Red campaigns should be avoided until restocked")

📅 MULTI-PERIOD WINNERS ANALYSIS WITH STOCK AVAILABILITY
📊 Analysis Date: October 08, 2025 at 12:12:01
🎯 Period Analysis: Last 7, 14, 21, and 30 days
📦 Stock Status: Based on Detailed Stock List (Column B)
🏆 Winner Logic: 60% Conversion + 40% Sales (filtered by period)

📦 Processing Stock Data...
✅ Stock data processed: 4714 unique items
📊 Stock range: 1 - 14118 bottles
📦 STOCK STATUS LEGEND:
🟢 Green: 11+ bottles (good for resending campaigns)
🟡 Yellow: 1-10 bottles (proceed with caution)
🔴 Red: 0 bottles (avoid resending)
⚪ White: Unknown status (item not found in stock list)

🎨 PRICE TIER LEGEND:
🟣 Purple: Extra Luxury (1000+ CHF) | 🟨 Gold: Luxury (500+ CHF)
🟦 Blue: Premium (150+ CHF) | 🩷 Pink: Mid-Range (80+ CHF) | 🟢 Green: Budget (<80 CHF)


🗓️ LAST 7 DAYS ANALYSIS
--------------------------------------------------
🏆 TOP 10 WINNERS - LAST 7 DAYS:


,Period_Rank,🎨,📦,Campaign_No,Wine,Vintage,Starting_Date,Total_Sales_Amount_LCY,Conversion_Rate_%,Weighted_Score,Stock_Status,Main_Item_No
0,1,🟨,🟢,CM-25-02630,IX Proprietary Red Estate,2022,2025-10-02,82'723.98,4.94,1.0000,Good (222 bottles),63171
1,2,🟨,🟡,CM-25-02637-D,Hommage à Jacques Perrin,2013,2025-10-03,1'587.81,3.57,0.4413,Caution (1 bottles),32234
2,3,🟦,🟢,CM-25-02629,Lynch Bages,2009,2025-10-01,48'449.89,0.87,0.3399,Good (1428 bottles),2314
3,4,🟦,⚪,CM-25-02628-D,Grillet Blanc,2019,2025-10-02,6'674.44,2.03,0.2788,Unknown status,62552
5,5,🟢,🟢,CM-25-02687,Rieussec La Metamorphose,2019,2025-10-07,11'431.52,1.52,0.2399,Good (212 bottles),54824
7,6,🟢,🟢,CM-25-02645,Clos Haut Peyraguey,2017,2025-10-03,1'398.32,1.83,0.2290,Good (96 bottles),45585
8,7,🟦,⚪,CM-25-02644,Yquem,2015,2025-10-03,18'809.76,0.56,0.1590,Unknown status,36554
10,8,🟦,🟢,CM-25-02633,La Fleur Pétrus,2018,2025-10-02,15'094.03,0.56,0.1410,Good (180 bottles),49027
11,9,🩷,⚪,CM-25-02646,Champagne Brut Comtes de Champagne Blanc de Bl...,2014,2025-10-03,1'460.61,0.60,0.0799,Unknown status,65733
12,10,🟢,⚪,CM-25-02680,Faugères,2020,2025-10-06,3'880.10,0.28,0.0528,Unknown status,53992



📊 LAST 7 DAYS SUMMARY:
• Total campaigns in period: 21
• Top 10 campaigns analyzed
• Average weighted score: 0.2962
• Total sales (Top 10): CHF 191'510.45

📦 STOCK DISTRIBUTION (Top 10):
   🟢 Good for resending: 5 campaigns (50.0%)
   🟡 Proceed with caution: 1 campaigns (10.0%)
   ⚪ Unknown status: 4 campaigns (40.0%)

📅 LAST 14 DAYS ANALYSIS
--------------------------------------------------
🏆 TOP 10 WINNERS - LAST 14 DAYS:


,Period_Rank,🎨,📦,Campaign_No,Wine,Vintage,Starting_Date,Total_Sales_Amount_LCY,Conversion_Rate_%,Weighted_Score,Stock_Status,Main_Item_No
0,1,🟨,🟢,CM-25-02630,IX Proprietary Red Estate,2022,2025-10-02,82'723.98,4.94,1.0000,Good (222 bottles),63171
1,2,🟨,🟡,CM-25-02637-D,Hommage à Jacques Perrin,2013,2025-10-03,1'587.81,3.57,0.4413,Caution (1 bottles),32234
2,3,🟦,🟢,CM-25-02629,Lynch Bages,2009,2025-10-01,48'449.89,0.87,0.3399,Good (1428 bottles),2314
3,4,🟦,⚪,CM-25-02628-D,Grillet Blanc,2019,2025-10-02,6'674.44,2.03,0.2788,Unknown status,62552
4,5,🟦,⚪,CM-25-02625-D,El Tamboril Blanco,2016,2025-10-01,9'253.45,1.76,0.2585,Unknown status,65647
5,6,🟢,🟢,CM-25-02687,Rieussec La Metamorphose,2019,2025-10-07,11'431.52,1.52,0.2399,Good (212 bottles),54824
6,7,🩷,🟢,CM-25-02624,Tignanello,2022,2025-10-01,23'211.00,1.00,0.2337,Good (318 bottles),63038
7,8,🟢,🟢,CM-25-02645,Clos Haut Peyraguey,2017,2025-10-03,1'398.32,1.83,0.2290,Good (96 bottles),45585
8,9,🟦,⚪,CM-25-02644,Yquem,2015,2025-10-03,18'809.76,0.56,0.1590,Unknown status,36554
9,10,🟨,🟡,CM-25-02582,Champagne Brut Dom Pérignon,2013,2025-10-01,17'613.31,0.48,0.1435,Caution (3 bottles),63033



📊 LAST 14 DAYS SUMMARY:
• Total campaigns in period: 25
• Top 10 campaigns analyzed
• Average weighted score: 0.3324
• Total sales (Top 10): CHF 221'153.47

📦 STOCK DISTRIBUTION (Top 10):
   🟢 Good for resending: 5 campaigns (50.0%)
   🟡 Proceed with caution: 2 campaigns (20.0%)
   ⚪ Unknown status: 3 campaigns (30.0%)

📆 LAST 21 DAYS ANALYSIS
--------------------------------------------------
🏆 TOP 10 WINNERS - LAST 21 DAYS:


,Period_Rank,🎨,📦,Campaign_No,Wine,Vintage,Starting_Date,Total_Sales_Amount_LCY,Conversion_Rate_%,Weighted_Score,Stock_Status,Main_Item_No
0,1,🟨,🟢,CM-25-02630,IX Proprietary Red Estate,2022,2025-10-02,82'723.98,4.94,1.0000,Good (222 bottles),63171
1,2,🟨,🟡,CM-25-02637-D,Hommage à Jacques Perrin,2013,2025-10-03,1'587.81,3.57,0.4413,Caution (1 bottles),32234
2,3,🟦,🟢,CM-25-02629,Lynch Bages,2009,2025-10-01,48'449.89,0.87,0.3399,Good (1428 bottles),2314
3,4,🟦,⚪,CM-25-02628-D,Grillet Blanc,2019,2025-10-02,6'674.44,2.03,0.2788,Unknown status,62552
4,5,🟦,⚪,CM-25-02625-D,El Tamboril Blanco,2016,2025-10-01,9'253.45,1.76,0.2585,Unknown status,65647
5,6,🟢,🟢,CM-25-02687,Rieussec La Metamorphose,2019,2025-10-07,11'431.52,1.52,0.2399,Good (212 bottles),54824
6,7,🩷,🟢,CM-25-02624,Tignanello,2022,2025-10-01,23'211.00,1.00,0.2337,Good (318 bottles),63038
7,8,🟢,🟢,CM-25-02645,Clos Haut Peyraguey,2017,2025-10-03,1'398.32,1.83,0.2290,Good (96 bottles),45585
8,9,🟦,⚪,CM-25-02644,Yquem,2015,2025-10-03,18'809.76,0.56,0.1590,Unknown status,36554
9,10,🟨,🟡,CM-25-02582,Champagne Brut Dom Pérignon,2013,2025-10-01,17'613.31,0.48,0.1435,Caution (3 bottles),63033



📊 LAST 21 DAYS SUMMARY:
• Total campaigns in period: 25
• Top 10 campaigns analyzed
• Average weighted score: 0.3324
• Total sales (Top 10): CHF 221'153.47

📦 STOCK DISTRIBUTION (Top 10):
   🟢 Good for resending: 5 campaigns (50.0%)
   🟡 Proceed with caution: 2 campaigns (20.0%)
   ⚪ Unknown status: 3 campaigns (30.0%)

🗓️ LAST 30 DAYS ANALYSIS
--------------------------------------------------
🏆 TOP 10 WINNERS - LAST 30 DAYS:


,Period_Rank,🎨,📦,Campaign_No,Wine,Vintage,Starting_Date,Total_Sales_Amount_LCY,Conversion_Rate_%,Weighted_Score,Stock_Status,Main_Item_No
0,1,🟨,🟢,CM-25-02630,IX Proprietary Red Estate,2022,2025-10-02,82'723.98,4.94,1.0000,Good (222 bottles),63171
1,2,🟨,🟡,CM-25-02637-D,Hommage à Jacques Perrin,2013,2025-10-03,1'587.81,3.57,0.4413,Caution (1 bottles),32234
2,3,🟦,🟢,CM-25-02629,Lynch Bages,2009,2025-10-01,48'449.89,0.87,0.3399,Good (1428 bottles),2314
3,4,🟦,⚪,CM-25-02628-D,Grillet Blanc,2019,2025-10-02,6'674.44,2.03,0.2788,Unknown status,62552
4,5,🟦,⚪,CM-25-02625-D,El Tamboril Blanco,2016,2025-10-01,9'253.45,1.76,0.2585,Unknown status,65647
5,6,🟢,🟢,CM-25-02687,Rieussec La Metamorphose,2019,2025-10-07,11'431.52,1.52,0.2399,Good (212 bottles),54824
6,7,🩷,🟢,CM-25-02624,Tignanello,2022,2025-10-01,23'211.00,1.00,0.2337,Good (318 bottles),63038
7,8,🟢,🟢,CM-25-02645,Clos Haut Peyraguey,2017,2025-10-03,1'398.32,1.83,0.2290,Good (96 bottles),45585
8,9,🟦,⚪,CM-25-02644,Yquem,2015,2025-10-03,18'809.76,0.56,0.1590,Unknown status,36554
9,10,🟨,🟡,CM-25-02582,Champagne Brut Dom Pérignon,2013,2025-10-01,17'613.31,0.48,0.1435,Caution (3 bottles),63033



📊 LAST 30 DAYS SUMMARY:
• Total campaigns in period: 25
• Top 10 campaigns analyzed
• Average weighted score: 0.3324
• Total sales (Top 10): CHF 221'153.47

📦 STOCK DISTRIBUTION (Top 10):
   🟢 Good for resending: 5 campaigns (50.0%)
   🟡 Proceed with caution: 2 campaigns (20.0%)
   ⚪ Unknown status: 3 campaigns (30.0%)

🎯 MULTI-PERIOD ANALYSIS COMPLETE
📊 Use the stock status indicators to make resending decisions
🟢 Green campaigns are ideal for immediate resending
🟡 Yellow campaigns should be monitored closely
🔴 Red campaigns should be avoided until restocked
